In [ ]:
# https://github.com/ssut/py-hanspell/blob/master/README.md 

In [ ]:
%pip install aiohttp

In [ ]:
%pip install py-aiohanspell

In [14]:
from aiohanspell import spell_checker 
results = await spell_checker.check(u"안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.")
print(results.as_dict()) 
results.checked      


{'result': True, 'original': '안녕 하세요. 저는 한국인 입니다. 이문장은 한글로 작성됬습니다.', 'checked': '안녕하세요. 저는 한국인입니다. 이 문장은 한글로 작성됐습니다.', 'errors': 4, 'words': OrderedDict([('안녕하세요.', 2), ('저는', 0), ('한국인입니다.', 2), ('이', 2), ('문장은', 2), ('한글로', 0), ('작성됐습니다.', 1)]), 'time': 0.17728519439697266}
<class 'aiohanspell.response.Checked'>


In [ ]:
### 마르코프 체인  
## 순서 ## 
# 1. 문장을 단어로 분할(형태소 분석)합니다. 
# 2. 단어의 전후 연결을 딕셔너리에 등록합니다. 
# 3. 사전을 사용해 임의 문장을 생성합니다. 


In [24]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Okt 
from aiohanspell import spell_checker # 네이버 맞춤법 검사기 라이브러리
import os, re, json, random

# 마르코프 체인 딕셔너리 만들기     
def make_dic(words):
  tmp = ["@"] 
  dic = {}
  for word in words: 
    tmp.append(word)
    if len(tmp) < 3: continue
    if len(tmp) > 3: tmp = tmp[1:] 
    set_word3(dic, tmp) # 딕서녀리에 데이터를 등록
    if word == ".": 
      tmp = ["@"]
      continue 
  return dic 

# 딕셔너리에 데이터 등록하기 
def set_word3(dic, s3): 
  w1, w2, w3 = s3
  if not w1 in dic: dic[w1] = {}
  if not w2 in dic[w1]: dic[w1][w2] = {}
  if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
  dic[w1][w2][w3] += 1

# 임의의 문장 생성 
def make_sentence(dic): 
  ret = []
  if not "@" in dic: return "no dic" # 딕셔너리가 비어있다는 뜻
  top = dic["@"]
  w1 = word_choice(top)
  w2 = word_choice(top[w1]) 
  ret.append(w1) 
  ret.append(w2)
  while True:
    w3 = word_choice(dic[w1][w2])
    ret.append(w3)
    if w3 == ".": break
    w1, w2 = w2, w3
  ret = "".join(ret)

  return ret ## 맞춤법 검사는 메인에서 처리 


def word_choice(sel):
  keys = sel.keys()
  return random.choice(list(keys))  


# 문장 읽어 들이기 
toji_file = "toji.txt"
dict_file = "markov-toji.json"

if not os.path.exists(dict_file): 
#토지 텍스트 파일 읽어 들이기
  fp = codecs.open("data/BEXX0003.txt", "r", encoding="utf-16")
  soup = BeautifulSoup(fp, "html.parser")
  body = soup.select_one("body > text")
  text = body.getText()
  text = text.replace("...", "") # 현재 koNLPy가 ...을 구두점으로 잡지 못하는 문제 임시 해결 
  
  # 형태소 분석
  okt = Okt()
  malist = okt.pos(text, norm=True)
  words = []
  for word in malist:
    # 구두점 등은 대상에서 제외(단 마침표는 포함) 
    if not word[1] in ["Punctuation"]:
      words.append(word[0]) 
    if word[0] == ".":
      words.append(word[0])
  # 딕셔너리 생성 
  dic = make_dic(words)
  json.dump(dic, open(dict_file,"w", encoding="utf-8")) 
else:
  dic = json.load(open(dict_file,"r"))

# 문장 만들기
for i in range(3):
  s = make_sentence(dic) 
  t = await spell_checker.check(s)
  print(t.checked)
  print("-" *30)  


몹시 흥분한 줄 알았는데 뜻밖에 머슴살이를 부탁하는 건 아니더라만성미를 익히 알고 아가리에 밥이 그리 대단했을 것도 없지요여기서 뭘 하는 거야서희를 흔들어 보인다.
------------------------------
긴 그림자가 흔들렸다.
------------------------------
대신 김서방이 죽으믄 고만이 다 알아도 모르는 거는 나다 이 년은 만나는 쪽쪽 사내를 홀 타냈더라고 그거 다 망했다 마.
------------------------------


In [25]:
from pprint import pprint
import json 
dic = json.load(open("markov-toji.json", "r")) 


In [39]:
print(dic['걱정'])

{'이며': {'이윽고': 1}, '마라': {'\n': 3, '.': 3}, '은': {'마라': 2, '안': 1}, '할': {'것': 1}, '말고': {'마저': 1, '처': 1}, '들': {'을': 1}, '을': {'다': 1}, '이': {'있고': 1, '있더라고': 1, '없을': 1, '요': 1}, '말': {'게': 1}, '일세': {'.': 1}, '하실': {'것': 1}, '이고': {'송장': 1, '\n': 1}, '이제': {'.': 1}, '이다': {'그': 1}}


In [3]:
import tensorflow as tf
import codecs
from tensorflow import keras
from bs4 import BeautifulSoup
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout 
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
from keras.utils.data_utils import get_file 
import numpy as np 
import random, sys

Using TensorFlow backend.


In [4]:
fp = codecs.open("data/BEXX0003.txt", "r", encoding="utf-16") 
soup = BeautifulSoup(fp, "html.parser")
body = soup.select_one("body")
text=body.getText() + " "
print('코퍼스의 길이: ', len(text))  

코퍼스의 길이:  311682


In [ ]:

# 문자를 하나하나 읽어 들이고 ID 붙이기
chars = sorted(list(set(text)))
print('사용되고 있는 문자의 수 :', len(chars)) 
char_indices = dict((c, i) for i, c in enumerate(chars)) 
indices_char = dict((i, c)for i, c in enumerate(chars))#ID–문자 

# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기 
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen]) 
  next_chars.append(text[i + maxlen])
print('학습할 구문의 수 :', len(sentences)) 
print('텍스트를 ID 벡터로 변환합니다...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
Y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence): 
    X[i, t, char_indices[char]] = 1
    Y[i, char_indices[next_chars[i]]] = 1


# 모델 구축하기(LSTM) print('모델을 구축합니다...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)))) 
model.add(Dense(len(chars)))
model.add(Activation('softmax')) 
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer) 

 # 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64') 
  preds = np.log(preds) / temperature 
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds) 
  probas = np.random.multinomial(1, preds, 1) 
  return np.argmax(probas)
# 학습시키고 텍스트 생성하기 반복 
for iteration in range(1, 60):  
  print()
  print('-' * 50)
  print('반복 = ', iteration)
  model.fit(X, Y, batch_size=128, nb_epoch=1) 
# 임의의 시작 텍스트 선택하기
  start_index = random.randint(0, len(text) - maxlen - 1)
   # 다양한 다양성의 문장 생성
  for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('-- 다양성 = ', diversity) 
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('- 115 = "' + sentence + '"') 
    sys.stdout.write(generated)
# 시드를 기반으로 텍스트 자동 생성 
    for i in range(400):
      x = np.zeros((1, maxlen, len(chars))) 
      for t, char in enumerate(sentence): 
        x[0, t, char_indices[char]] = 1.
# 다음에 올 문자를 예측하기
      preds = model.predict(x, verbose=0)[0]
      next_index = sample(preds, diversity)
      next_char = indices_char[next_index] 
      # 출력하기
      generated += next_char
      sentence = sentence[1:] + next_char
      sys.stdout.write(next_char)
      sys.stdout.flush() 
      
    print()

In [39]:
!chmod +x cgi-bin/chatbot.py

chmod: /../cgi-bin/chatbot.py: No such file or directory


In [ ]:
import cgi
from botengine import make_reply

 # 입력 양식의 글자 추출하기  
form = cgi.FieldStorage()
def main():
  m = form.getvalue("m", default="")
  if m == "" : show_form() 
  elif m == "say" : api_say()

# 사용자의 입력에 응답하기 
def api_say():
  print("Content-Type: text/plain; charset=utf-8") 
  print("")
  txt = form.getvalue("txt", default="")
  if txt == "": return
  res = make_reply(txt)
  print(res)
# 입력 양식 출력하기
def show_form():
  print("Content-Type: text/html; charset=utf-8") 
  print("")
  print("""
<html>meta charset="utf-8"><body>
<script src="https://code.jquery.com/jquery-3.1.1.min.js"></script> 
<style>
  h1 { background-color: #ffe0e0; }
  div { padding:10px; }
  span { border-radius: 10px; background-color: #ff0e0; padding:8px; }
  .bot { text-align: left; }
  .usr { text-align: right; } 
</style>
<h1>대화 하기</h1>
<div id="chat"></div>
<div class='usr'><input id="txt" size="40">
<button onclick="say()">전송</button></div>
<script>
var url = "./chatbot.py"; 
function say() {
  var txt = $('#txt').val();
  $.get(url, {"m":"say","txt":txt},
  function(res) {
  var html = <div class='usr'><span>" + esc(txt) +
    "</span>: 나<div><div class='bot'> 봇: <span>" +
    esc(res) + "</span></div>"; 
    $('#chat').html($('#chat').html()+html); 
    $('#txt').val('').focus();
  });
}
function esc(s) {
  return s.replace('&', '&amp;').replace('<','&lt;')
          .replace('>', '&gt;"); 
}          
</script></body></html>
""") 

main()



In [ ]:
import codecs
from bs4 import BeautifulSoup
import urllib.request
from konlpy.tag import Okt
import os, re, json, random
dict_file = "chatbot-data.json"
dic = {}
okt = Okt()
# 딕셔너리에 단어 등록하기 - (※1) 
def register_dic(words):
  global dic
  if len(words) == 0: return
  tmp = ["@"]
  for i in words:
    word = i[0]
    if word == "" or word == "\r\n" or word == "\n": continue
    tmp.append(word)
    if len(tmp) <3: continue
    if len(tmp) > 3: tmp = tmp[1:]
    set_word3(dic, tmp)
    if word == "." or word == "?":
      tmp =["@"] 
      continue
    # 딕셔너리가 변경될 때마다 저장하기
  json.dump(dic, open(dict_file,"w", encoding="utf-8"))

# 딕셔너리에 글 등록하기 
def set_word3(dic, s3): 
  w1, w2, w3 = s3
  if not w1 in dic: dic[w1] = {}
  if not w2 in dic[w1]: dic[w1][w2] = {}
  if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
  dic[w1][w2][w3] += 1

# 문장 생성 
def make_sentence(dic): 
  ret = []
  if not "@" in dic: return "no dic" # 딕셔너리가 비어있다는 뜻
  top = dic["@"]
  w1 = word_choice(top)
  w2 = word_choice(top[w1]) 
  ret.append(w1) 
  ret.append(w2)
  while True:
    if w1 in dic and w2 in dic[w1]: 
      w3 = word_choice(dic[w1][w2])
    else:
      w3 = ""
    ret.append(w3)
    if w3 == "." or w3 == "?" or w3 == "": break
    w1, w2 = w2, w3
  ret = "".join(ret)

  return ret 

def word_choice(sel):
  keys = sel.keys()
  return random.choice(list(keys))  
